<a href="https://colab.research.google.com/github/jinsusong/21-study-tensorflow/blob/main/%EC%98%81%EC%96%B4%20%ED%85%8D%EC%8A%A4%ED%8A%B8%20%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 텐서플로2와 머신러닝으로 시작하는 자연어 처리 
### 로지스틱 회귀부터 BERT와 GPT2 까지


### 텍스트 분류



: 영어 텍스트 분류 , 한글 텍스트 분류 샘플 
- 감정분석  
- 해당 모델을 통해 새로운 리뷰가 긍정적인 리뷰인지 부정적인 리뷰인지 예측

: 데이터 이름 : Bag Words Meets Bags of Popcorn
: 데이터 용도 : 텍스트 분류 학습을 목적으로 사용
: 데이터 권한 : MIT 권한을 가지고 있으나 캐글에 가입한 후 사용하길 권장





워드 팝콘 : 영화 데이터베이스에서 나온 영화 평점 데이터를 활용한 캐글 문제 

목표 
1. 데이터 불러오기 , 전터리하기 
2. 데이터 분석하기 
3. 문제 해결을 위한 알고리즘 모델링 

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [ ]:
from google.colab import files

uploaded = files.upload() 

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))


데이터 불러오기

In [ ]:
import pandas as pd
import io
train_data = pd.read_csv(io.StringIO(uploaded['labeledTrainData.tsv'].decode('utf-8')), header=0, delimiter="\t",quoting=3)
print(train_data)

In [ ]:
train_data.head()

데이터분석 진행 
 - 데이터 크기 
 - 데이터의 개수 
 - 각 리뷰의 문자 길이 분포
 - 많이 사용된 단어
 - 긍정, 부정 데이터의 분포 
 - 각 리뷰의 단어 개수 분포
 - 특수문자 및 대문자, 소문자 비율
 


In [ ]:
print("파일 크기 : ")
for file in os.listdir():
    if 'tsv' in file and 'zip' not in file:
        print(file.ljust(30) + str(round(os.path.getsize(file)/1000000,2))+ 'MB')
        

In [ ]:
print('전체 학습 데이터의 개수 : {}'.format(len(train_data)))


리뷰 데이터 길이 확인

In [ ]:
train_length = train_data['review'].apply(len)
train_length.head()

리뷰의 길이 변수를 통해 히스토그램 그리기 

In [ ]:
plt.figure(figsize=(12,5))
plt.hist(train_length, bins=200, alpha=0.5, color='r', label='word')
plt.yscale('log',nonposy='clip')
plt.title ('Log Histogram of length of review')
plt.xlabel('Length of review')
plt.ylabel('Number of review')

리뷰 길이에 대한 통계값 확인하기 


In [ ]:
print('리뷰 길이 최댓값 : {}'.format(np.max(train_length)))
print('리뷰 길이 최솟값 : {}'.format(np.min(train_length)))
print('리뷰 길이 평균값 : {:.2f}'.format(np.mean(train_length)))
print('리뷰 길이 표준편차 : {:.2f}'.format(np.std(train_length)))
print('리뷰 길이 중간값 : {}'.format(np.median(train_length)))

print('리뷰 길이 제 1사분위 : {}'.format(np.percentile(train_length,25)))
print('리뷰 길이 제 3사분위 : {}'.format(np.percentile(train_length,75)))

박스 플롯 그리기 

In [ ]:
plt.figure(figsize=(12,5))
plt.boxplot(train_length,
            labels=['counts'],
            showmeans=True)

리뷰에 많이 사용된 단어 워드클라우드로 확인하기

In [ ]:
#!pip install wordcloud
from wordcloud import WordCloud
cloud = WordCloud(width=800, height=600).generate(" ".join(train_data['review']))
plt.figure(figsize=(20,15))
plt.imshow(cloud)
plt.axis('off')












라벨의 분포를 확인해본다. 

In [ ]:
fig, axe = plt.subplots(ncols=1)
fig.set_size_inches(6,3)
sns.countplot(train_data['sentiment'])

In [ ]:
print('긍정 리뷰 개수 : {}'.format(train_data['sentiment'].value_counts()[1]))
print('부정 리뷰 개수 : {}'.format(train_data['sentiment'].value_counts()[0]))


이제 각 리뷰를 단어 기준으로 나눠서 각 리뷰당 단어의 개수를 확인한다. 
단어는 띄어쓰기 기준으로 하나의 단어라 생각하고 개수를 계산한다. 


In [ ]:
train_word_counts = train_data['review'].apply(lambda x : len(x.split(' ')))

In [ ]:
plt.figure(figsize=(15,10))
plt.hist(train_word_counts, bins=50, facecolor='r',label='train')
plt.title('Log-Histogram of word count in review', fontsize=15)
plt.yscale('log',nonposy='clip')
plt.legend()
plt.xlabel('Number of words',fontsize=15)
plt.ylabel('Number of reviews',fontsize=15)



In [ ]:
print('리뷰 단어 최댓값 : {}'.format(np.max(train_word_counts)))
print('리뷰 단어 최솟값 : {}'.format(np.min(train_word_counts)))
print('리뷰 단어 평균값 : {:.2f}'.format(np.mean(train_word_counts)))
print('리뷰 단어 표준편차 : {:.2f}'.format(np.std(train_word_counts)))
print('리뷰 단어 중간값 : {}'.format(np.median(train_word_counts)))

print('리뷰 단어 제 1사분위 : {}'.format(np.percentile(train_word_counts,25)))
print('리뷰 단어 제 3사분위 : {}'.format(np.percentile(train_word_counts,75)))

데이터 전처리 

In [ ]:
import re
import pandas
import numpy
import json
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer



In [ ]:
#train_data = pd.read_csv(io.StringIO(uploaded['labeledTrainData.tsv'].decode('utf-8')), header=0, delimiter="\t",quoting=3)
print(train_data)

In [ ]:
print(train_data['review'][0])

In [ ]:
review = train_data['review'][0] # 리뷰 중 하나 가져오기 
review_text = BeautifulSoup(review, "html5lib").get_text()# html 태그 제거 
review_text = re.sub("[^a-zA-Z]"," ", review_text) # 영어 문자를 제외한 나머지는 모두 공백으로 바꾼다. 



In [ ]:
print(review_text)

불용어 제거

In [ ]:
stop_words = set(stopwords.words('english'))

review_text = review_text.lower()
words = review_text.split()
words = [w for w in words if not w in stop_words]


In [ ]:
def preprocessing(review, remove_stopwords= False):

    review_text = BeautifulSoup(review, "html5lib").get_text()

    review_text = re.sub("[^a-zA-Z]", " ", review_text)

    words = review_text.lower().split()

    if remove_stopwords:

        stops = set(stopwords.words("english"))
        words=[w for w in words if not w in stops]
        clean_review = ' '.join(words)

    else:
        clean_review = ' '.join(words)

    return clean_review

clean_train_reviews = []
for review in train_data['review']:
    clean_train_reviews.append(preprocessing(review, remove_stopwords = True)
    

clean_train_reviews[0]

    
    
    
    
    
    
    
    
    
    
    
    )